In [1]:
def give_list(query = "Tata Steels", class_name_1="JtKRv", class_name_2="hvbAAd"):
    import requests
    from bs4 import BeautifulSoup
    import time
    
    company_name = query
    company_name_processed = company_name.replace(" ", "%20")
    
    url = f'https://news.google.com/search?q={company_name_processed}&hl=en-IN&gl=IN&ceid=IN%3Aen'
    
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    class_name = class_name_1
    temp = soup.find_all(class_=class_name)
    published_at = soup.find_all(class_=class_name_2)
    
    result_list = []
    
    for t, p in zip(temp, published_at):
        # Split the time string into value and unit
        time_string = p.text[:-4]
        parts = time_string.split(maxsplit=1)
        
        if len(parts) == 2:
            value, unit = parts
            # Convert the numeric value to an integer (if possible)
            try:
                numeric_value = int(value)
            except ValueError:
                # Handle cases where the value cannot be converted to an integer
                numeric_value = None
        else:
            # Handle cases where the time string doesn't match the expected format
            numeric_value = None
            unit = time_string
        
        pair = {"Time": [numeric_value, unit], "Context": t.text}
        result_list.append(pair)
    return result_list

def tempo(classifier, result_list):
    score_list = []
    for i in result_list:
        score_list.append(classifier.predict(i['Context']))
    dict = {
        'negative': -1,
        'neutral': 0,
        'positive': 1
    }
    score = []
    for i in score_list:
        temp = i[0]['label']
        temp = dict[temp]
        score.append(temp)
    total_score = sum(score) 
    return score_list,total_score

In [2]:
from datasets import load_dataset

from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification

model_name = "mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
classifier = pipeline("sentiment-analysis",model=model, tokenizer= tokenizer)


Device set to use cuda:0


In [6]:
input_query = input()
result_list = give_list(query = input_query)
temp,temp_score = tempo(classifier, result_list)

print(f'Score for {input_query} = {temp_score}')
# print("Some random labels \n",temp[:5])

Score for Tesla = -31
